In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
import torch.nn as nn

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [6]:
import nltk
from nltk.tokenize import sent_tokenize

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import textwrap

In [93]:
device = torch.device("cuda:1")
model_name = 'naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B' #-1.5B'
model = AutoModelForCausalLM.from_pretrained(model_name)

In [94]:
model = model.half()
model = model.to(device)

In [95]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [96]:
max_token_length = getattr(model.config, "max_position_embeddings", None) #getattr(tokenizer, "model_max_length", None)

#if max_token_length is None or max_token_length > 1e5:  # 무한대 설정인 경우
#            max_token_length = 4096   # 일반 LLM 기본값 가정

In [97]:
# 생성할 토큰 수 여유 확보
gen_new_tokens = 256
prompt_max = max(16, max_token_length - gen_new_tokens)

In [98]:
prompt_max

7936

In [99]:
def get_summarize(text, ksic='어떤 산업') :
    role = f'너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 {ksic}에 대한 뉴스 기사인데, 이것을 요약해줘.'
    #print(role)
    
    chat = [
              {"role": "tool_list", "content": ""},
              {"role": "system", "content": role}, 
              {"role": "user", "content": text},
    ]
    
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_dict=True, return_tensors="pt", 
                                          truncation=True, max_length=prompt_max)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    output_ids = model.generate(**inputs, max_length=max_token_length, stop_strings=["<|endofturn|>", "<|stop|>"], tokenizer=tokenizer)
    
    responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    
    _, _, answer = responses[0].partition("assistant\n")

    return answer.strip()

In [77]:
df = pd.read_csv('summary_output.tsv', sep='\t')

In [78]:
len(df)

765026

In [79]:
df

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."
...,...,...,...,...,...,...
765021,NWRW1900000038.41782,T,네이버·다음 실시간 검색어 순위 ‘정부 요청 시 삭제’ 규정 운영했다 ㆍ자동완성·연...,NaN,0.000000,국내 양대 포털사이트인 네이버와 카카오(다음)가 정부 당국이 요청할 경우 실시간 급...
765022,NLRW2000000006.29284,T,"[미래사회포럼]최정준 동방문화대학원대 교수, ""소명 알고 완성하는게 천명 이루는 길...",NaN,0.000000,22일 열린 미래사회포럼 강연자로 나선 최정준 동방문화대학원대학교 미래예측학과 교수...
765023,NLRW2000000006.29247,T,"원더브라88원, 8월 8일 기념해 초대형 프로모션 '행사 가격은?' 라이프스타일 전...",NaN,0.000000,라이프스타일 전문기업 그리티가 푸쉬업 브라브랜드 '원더브라' 브랜드데이를 맞아 8일...
765024,NWRW1900000038.41262,T,마이클 코스털리츠 방한 “남다른 시각과 국가적 투자가 노벨상 만들어” ㆍ올해 노벨물...,NaN,0.000000,올해 노벨 물리학상 수상자 중 한 명인 마이클 코스털리츠 미국 브라운대 교수는 20...


In [80]:
df_best_scores = df[df['max_scores'] > 0.38]

In [81]:
df_best_scores

,id,topic,text,ksic,max_scores,summary_new
0,NLRW1900000024.16590,S,1221m 초콜릿 파이 “3만명이 함께 먹을 수 있어” 1221m 초콜릿 파이가 등...,c107,0.930233,최근 스위스의 한 지방 제빵사협회가 창립 25주년을 기념해 3만 명이 먹을 수 있는...
1,NWRW1900000009.21980,S,"고등어야, 어디 있니? 어획량 20년 새 3분의 1로 국내산 고등어 어획량이 20년...",c102,0.718052,국회 농림축산식품해양수산위원회 박완주 의원(더불어민주당)이 수협중앙회에서 제출받은 ...
2,NLRW1900000144.10274,S,"고령 우곡 농민들 ""올해 수박농사 빈손…막막할 뿐"" 이상기후 물량 줄고 품질 떨어져...",c103,0.639269,고령군 우곡면 답곡1리 김진수씨(여·72)의 수박 비닐하우스. 30여년째 수박농사를...
3,NLRW2200000008.12137,S,"태안군, 해풍을 맞고 자란 명품 양파 수확 ‘한창!’ 태안의 비옥한 황토에서 갯바람...",c103,0.612410,태안 지역에는 약 150여 농가가 76ha 면적에서 양파를 재배하고 있으며 연간 3...
4,NWRW1900000011.9505,S,친환경 농산물 약진… 생산량 10% 넘어 국내에서 생산되는 농산물 10개 가운데 1...,c103,0.588431,"국내 생산되는 농산물 10개 가운데 1개 이상이 친환경 농산물인 것으로 나타났으며,..."
...,...,...,...,...,...,...
382785,NWRW1900000001.25080,T,삼성엔… 고맙고도 두려운 '아이폰 대박' 삼성전자가 애플의 '아이폰' 때문에 웃고 ...,c264,0.380656,삼성전자가 애플의 '아이폰' 때문에 웃고 울고 있는데 애플의 아이폰은 지난 2분기 ...
382786,NWRW1900000010.5442,T,"삼성 갤럭시S9, 애플 아이폰에 완승… 유럽 6國 소비자 평가서 1위 차지 애플 아...",c264,0.380547,20일 업계에 따르면 삼성전자의 갤럭시S9 시리즈가 이탈리아·프랑스·포르투갈·스페인...
382787,NPRW1900000056.8650,T,갤럭시노트3로 스위스철도 발권 삼성전자 모바일 B2B(기업 간 거래) 사업 역량이 ...,c264,0.380265,삼성전자는 갤럭시노트3갤럭시S4를 비롯한 스마트폰과 갤럭시탭3 등 태블릿 디바이스가...
382788,NLRW2200000005.12347,T,"갤럭시S21 울트라5G, 모바일 월드 콩그레스2021서 ‘최고 스마트폰’ 선정 삼성...",c264,0.380160,삼성전자 갤럭시 S21 울트라 5G가 스페인 바르셀로나에서 개최된 모바일 월드 콩그...


In [100]:
df_best_scores['summary_naver_0.5B'] = df_best_scores['text'].progress_apply(get_summarize)

  0%|          | 0/984 [00:00<?, ?it/s]

/tmp/ipykernel_1754494/1580680420.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_best_scores['summary_naver_0.5B'] = df_best_scores['text'].progress_apply(get_summarize)


In [88]:
# 1) 문자열 길이 계산
df_best_scores["len_text"] = df_best_scores["text"].str.len().fillna(0).astype(int)
df_best_scores["len_summary_new"] = df_best_scores["summary_new"].str.len().fillna(0).astype(int)
df_best_scores["len_summary_naver_1_5B"] = (
    df_best_scores["summary_naver_1.5B"].str.len().fillna(0).astype(int)
)

/tmp/ipykernel_1754494/3085855141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_best_scores["len_text"] = df_best_scores["text"].str.len().fillna(0).astype(int)
/tmp/ipykernel_1754494/3085855141.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_best_scores["len_summary_new"] = df_best_scores["summary_new"].str.len().fillna(0).astype(int)
/tmp/ipykernel_1754494/3085855141.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [90]:
# 2) 비율 계산 (text 길이가 0이면 NaN으로 둠)
den = df_best_scores["len_text"].replace(0, np.nan)
df_best_scores["ratio_summary_new_vs_text"] = df_best_scores["len_summary_new"] / den
df_best_scores["ratio_summary_naver_1_5B_vs_text"] = (
    df_best_scores["len_summary_naver_1_5B"] / den
)

/tmp/ipykernel_1754494/4006183490.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_best_scores["ratio_summary_new_vs_text"] = df_best_scores["len_summary_new"] / den
/tmp/ipykernel_1754494/4006183490.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_best_scores["ratio_summary_naver_1_5B_vs_text"] = (


In [92]:
df_best_scores[['ratio_summary_new_vs_text', 'ratio_summary_naver_1_5B_vs_text']].describe()

,ratio_summary_new_vs_text,ratio_summary_naver_1_5B_vs_text
count,984.000000,984.000000
mean,0.434103,0.447387
std,0.119107,0.141842
min,0.191753,0.029877
25%,0.341722,0.345003
50%,0.429865,0.441755
75%,0.511315,0.540266
max,1.021978,0.971831


In [73]:
text = tmp['text']

In [74]:
text

'[CES 2019 결산 좌담회]"4차 산업혁명시대, 오픈이노베이션으로 대응하자" 미국 라스베이거스에서 8일(현지시간)부터 11일까지 열렸던 \'CES 2019\'는 한층 발전한 인공지능(AI)과 사물인터넷(IoT), 자율주행, 스마트시티 등 가까운 미래에 펼쳐질 첨단기술을 제시했다. 불과 수년 전까지만 해도 추상적 개념 단계에 그쳤던 기술이 실제 상용화 수준에 이르렀다. 기술 진화 속도가 빨라지면서 우리 삶도 한층 빠른 속도로 개선될 것으로 기대된다. 전자신문은 CES 메인 전시장인 \'라스베이거스 컨벤션센터(LVCC) 콘퍼런스룸에서 \'CES를 통해 본 ICT 산업 대전망\'을 개최하고 4차 산업혁명 시대에 우리나라 산업계 대책을 논의했다. 각 분야 전문가는 공통으로 오픈이노베이션을 강조했다. 오픈이노베이션은 기업에 필요한 기술과 자원을 외부에서 조달하고, 보유 자원을 다른 기업과 공유해 새로운 가치를 창출한다는 개념이다. 기술이 복잡화, 고도화되는 한편 기술 변화 속도는 빨라지면서 기업이 독자적으로 서비스, 제품, 기술을 개발하기 어려운 시대가 되면서다. 전문가들은 각 분야에서 강점을 가진 중소기업과 대기업이 협업해야 글로벌 경쟁력을 갖출 수 있다고 분석했다. ◆참석자(가나다순) △김상철 한글과컴퓨터 회장 △김영삼 전자부품연구원장 △남기만 한국반도체산업협회 부회장 △민동욱 엠씨넥스 대표 △서광현 한국디스플레이산업협회 부회장 △유정열 산업통상자원부 산업혁신성장실장 △이정원 세미솔루션 대표 △진홍 한국전자정보통신산업진흥회 부회장 △사회=김승규 전자신문 전자자동차유통부장 ◇사회(김승규 전자신문 전자자동차유통부장)=올해 CES에서도 새로운 기술과 제품이 대거 등장했다. 새해 기술 트렌드를 읽을 수 있었던 자리였다. CES 2019를 둘러본 소감이 궁금하다. ◇진홍(한국전자정보통신산업진흥회 부회장)=이번에 CES 행사를 처음 방문했다. 직접 눈으로 확인해보니 세계 기술이 한 곳에 다 모인 자리였다. 우리 산업계와 글로벌 기업 기술 수준을 가늠하고 향후 어떻게 대응

In [76]:
get_summarize(top1000.iloc[1]['text'], ksic='전자 부품 제조업')

너는 뉴스를 요약하는 요약 전문가야. 다음 문장은 전자 부품 제조업에 대한 뉴스 기사인데, 이것을 요약해줘.


'CES 2019에서 논의된 주요 기술 트렌드는 인공지능(AI), 사물인터넷(IoT), 자율주행, 스마트시티 등이었습니다. 이러한 기술들은 이미 상용화 단계에 이르렀으며, 전자 부품 제조업에서도 이 기술들이 적용되고 있습니다. 전문가들은 오픈 이노베이션을 통해 기업 간 협력과 자원 공유가 중요하다고 강조했습니다. 이는 기술 변화 속도가 빨라지면서 독자적인 개발이 어려운 상황에서 필수적입니다. 또한, 각 분야에서 강점을 가진 중소기업과 대기업의 협력이 글로벌 경쟁력을 확보하는 데 중요하다고 분석되었습니다. CES 2019는 이러한 기술 혁신과 산업 협력의 중요성을 강조하는 데 기여했습니다.'

In [50]:
# text 열의 문자열 길이 계산
df["text_len"] = df["text"].str.len()

# 길이 기준으로 내림차순 정렬 후 상위 5개 추출
top1000 = df.nlargest(1000, "text_len")

# 결과 출력
print(top1000[["text", "text_len"]])

                                                     text  text_len
697187  [소재부품 국산화, 다시 시작하자] <10>결산 좌담회 본지는 7월 1일 일본 정부...     10843
689300  [CES 2019 결산 좌담회]"4차 산업혁명시대, 오픈이노베이션으로 대응하자" 미...     10549
546824  돼지열병 창궐하자 “대박…” 비극을 기다린 사람들 비극도 돈이 되는 시대는 치사율 ...     10232
698436  [2019 신년기획]결산 좌담회 '4차 산업혁명 해법을 찾자' 대한민국을 휘감는 대...      9657
698445  ‘집단지성의 성채’ 위키백과, ‘지식정보의 편향’ 넘을 수 있을까 ‘개방과 공유’....      8992
...                                                   ...       ...
694242  끈질긴 '구청장 친인척' 보도, 지역 채널의 존재 이유 정부가 LG유플러스의 CJ헬...      4181
79775   발사 임박했다는 북한 미사일, 그 실체는? 북한이 미사일 발사 준비를 완료한 것으로...      4180
650577  [아주초대석] 김희관 더블미 대표 “현실로 나온 판타지 세상··· 할머니도 만들 수...      4180
698398  공인인증서만? 30년 낡은 '요금인가제도' 사라진다 오랜 기간 낡은 규제로 지적돼 ...      4180
761287  개인정보 안 주면 지도 업데이트 안돼~ '아이나비' 브랜드로 업계 1위를 차지하고 ...      4180

[1000 rows x 2 columns]


In [51]:
top1000['summary_naver_llm'] = top1000['text'].progress_apply(get_summarize)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [58]:
tmp = top1000.iloc[0]

In [60]:
tmp

id                                                NPRW2000000005.14858
topic                                                                T
text                 [소재부품 국산화, 다시 시작하자] <10>결산 좌담회 본지는 7월 1일 일본 정부...
ksic                                                              c303
max_scores                                                     0.01521
summary_new          일본 조치 이후 약 한 달간 소재부품 국산화와 공급망 다변화에 대한 관심이 커지면서...
text_len                                                         10843
summary_naver_llm    이 기사에서는 소재부품 국산화 현황과 문제점, 대안에 대해 논의하는 가운데, 특히 ...
Name: 697187, dtype: object

In [61]:
tmp.summary_naver_llm

'이 기사에서는 소재부품 국산화 현황과 문제점, 대안에 대해 논의하는 가운데, 특히 반도체, 디스플레이, 배터리 등 다양한 산업 분야에서 국산화의 필요성과 전략에 대해 다루고 있습니다. 일본 정부의 수출규제 이후 국산화 관심가 높아졌으며, 전문가들은 국산화의 필요성을 강조하고 있습니다. 정부는 소재부품 국산화를 위해 R&D 전략과 글로벌 협력 등을 강조하며, 중소기업과 대기업 간의 협력도 중요하다고 지적합니다. 또한, 기초과학 연구와 정부의 소재산업 육성 정책의 중요성을 언급하며, 장기적인 관점에서 국산화를 추진해야 한다고 강조합니다.'

In [62]:
tmp.summary_new

'일본 조치 이후 약 한 달간 소재부품 국산화와 공급망 다변화에 대한 관심이 커지면서 필요성이 더욱 부각되고 있다.이에 산학연관 전문가와 함께 우리 업계가 나아가야 할 방향을 찾는 결산 좌담회를 마련했다. 이신두(서울대 전기정보공학부 교수)=정부 연구개발(R&D) 사업에서 우리 희망사항과 실제로 할 수 있는 것이 무엇인지 면밀하게 들여다봐야 한다. 정부가 추가경정예산 2700억원 정도를 들여 국산화를 추진한다고 발표했다. 소재산업 독립을 도모하고 산업 자체 성장을 목적으로 삼아야 한다. 대기업과 중소기업 간 밸류 체인은 꽉 묶어놓으면 성장하지 못한다. 파트너십을 맺고 세계 시장으로 진입해야 한다. 그래서 국가가 전문 평가센터를 만들고 전략 소재를 모듈별로 테스트할 수 있도록 조성해야 한다."김창균=글로벌 수준의 우리 소재기업을 반드시 육성해야 한다"고 말했다. 과학기술에서 \'성과\'는 아웃풋이 아닌 \'인풋(투입)\'이라는 점을 알아야 한다. 대학과 기업을 잇는 국책연구소 역할이 절실하다. 제조혁신을 위한 플랫폼을 공유하고 상생하는 한국형 신성장 모델을 찾아야 한다. 제조업의 국내 개발과 자체 공급은 공급사와 수급기업 간 긴밀한 공조체계를 구축해 양산 능력을 지속 높여야 한다. 해외 글로벌 기업의 국내 유치는 연구소 수준이 아닌 국내 소재기업과 공동 개발생산까지 단계적으로 추진할 수 있게 법제도로 뒷받침해 실질적인 공동 연구개발을 해야 한다. 산업기술뿐 아니라 기초 연구, 목적 연구가 함께 균형 발전할 수 있어야 진짜 우리 만의 기술이 될 수 있다. 산업부가 산업전략을 제대로 수행하도록 힘을 실어줘야 한다. '

In [55]:
top1000.iloc[1]['summary_naver_llm']

"'CES 2019'에서 논의된 오픈이노베이션은 기업에 필요한 기술과 자원을 외부에서 조달하고, 보유 자원을 다른 기업과 공유하여 새로운 가치를 창출하는 개념입니다. 기술이 복잡하고 고도화됨에 따라 기업들이 독자적으로 서비스, 제품, 기술을 개발하기 어려워지면서 오픈이노베이션이 중요해졌습니다. 전문가들은 각 분야에서 강점을 가진 중소기업과 대기업이 협업해야 글로벌 경쟁력을 갖출 수 있다고 지적했습니다. 이를 통해 4차 산업혁명 시대에 대응하고 새로운 기회를 창출할 수 있을 것으로 기대됩니다."

In [44]:
get_summarize(top5.iloc[0]['text'])

'이 기사에서는 소재부품 국산화 현황과 문제점, 대안에 대해 논의하는 가운데, 특히 반도체, 디스플레이, 배터리 등 다양한 산업 분야에서 국산화의 필요성과 전략에 대해 다루고 있습니다. 일본 정부의 수출규제 이후 국산화 관심가 높아졌으며, 전문가들은 국산화의 필요성을 강조하고 있습니다. 정부는 소재부품 국산화를 위해 R&D 전략과 글로벌 협력 등을 강조하며, 중소기업과 대기업 간의 협력도 중요하다고 지적합니다. 또한, 기초과학 연구와 정부의 소재산업 육성 정책의 중요성을 언급하며, 장기적인 관점에서 국산화를 추진해야 한다고 강조합니다.'

In [47]:
top5['summary_naver_llm'] = top5['text'].progress_apply(get_summarize)

  0%|          | 0/5 [00:00<?, ?it/s]

In [48]:
top5

,id,topic,text,ksic,max_scores,summary_new,text_len,summary_naver_llm
697187,NPRW2000000005.14858,T,"[소재부품 국산화, 다시 시작하자] <10>결산 좌담회 본지는 7월 1일 일본 정부...",c303,0.015210,일본 조치 이후 약 한 달간 소재부품 국산화와 공급망 다변화에 대한 관심이 커지면서...,10843,"이 기사에서는 소재부품 국산화 현황과 문제점, 대안에 대해 논의하는 가운데, 특히 ..."
689300,NPRW2000000005.14771,T,"[CES 2019 결산 좌담회]""4차 산업혁명시대, 오픈이노베이션으로 대응하자"" 미...",c262,0.020692,미국 라스베이거스에서 8일(현지시간)부터 11일까지 열렸던 'CES 2019'는 한...,10549,'CES 2019'에서 논의된 오픈이노베이션은 기업에 필요한 기술과 자원을 외부에서...
546824,NWRW2000000002.25857,T,돼지열병 창궐하자 “대박…” 비극을 기다린 사람들 비극도 돈이 되는 시대는 치사율 ...,c101,0.070129,연천군 백학면의 양돈농장에서 아프리카돼지열병(ASF·이하 돼지열병)이 확진됐다. 전...,10232,아프리카돼지열병(AST)이 파주를 덮치면서 돼지열병을 테마로 삼은 투자자들과 기업들...
698436,NPRW2000000005.14769,T,[2019 신년기획]결산 좌담회 '4차 산업혁명 해법을 찾자' 대한민국을 휘감는 대...,c211,0.009373,4차 산업혁명은 제조업을 기반으로 고도 압축 성장을 거듭한 우리나라에 새로운 성장엔...,9657,2019 신년기획으로 열린 '4차 산업혁명 해법' 좌담회에서 전문가들은 대한민국이 ...
698445,NWRW2000000002.25495,T,"‘집단지성의 성채’ 위키백과, ‘지식정보의 편향’ 넘을 수 있을까 ‘개방과 공유’....",c272,0.008868,"최근 국내 통계물리학자들이 방대한 위키 빅데이터를 분석해, 적은 비율의 열성적 편집...",8992,"위키백과는 2001년 1월 15일 설립된 이후, 전 세계적으로 수많은 사용자와 편집..."


In [77]:
df.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')

In [78]:
df.iloc[0].text

'코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다. 의료 현장에서는 “코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다”는 목소리가 나온다. 18일 김범석 서울대 혈액종양내과 교수팀이 2019~2020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86% 대폭 증가한 것으로 나타났다. 말기 암 환자의 응급실 체류 시간도 늘었고, 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 52.3%에서 59.2%로 약 7%포인트 늘었다. 심폐소생술을 받은 암 환자도 12.5%에서 16.3%로 증가했다. 김 교수는 “코로나 환자도 중요하지만, 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다”고 말했다.'

In [26]:
run_summarize(df.iloc[0].text, tokenizer, model, device)

============ original ===============
코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코로나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나 확산으로 암 환자 등 일반 중환자 치료에 공백이 생기고 있다는 목소리가 나온다 18일 김범석 서울대 혈액종양내과 교수팀이 20192020년 서울대병원에서 사망한 암 환자 1456명을 분석한 결과 2019년 서울대병원 응급실에서 세상을 떠난 암 환자는 53명에서 2020년 99명으로 약 86 대폭 증가한 것으로 나타났다 말기 암 환자의 응급실 체류 시간도 늘었고 환자의 생존을 위해 강제로 혈압을 높이는 승압제를 사용한 환자 수도 523에서 592로 약 7포인트 늘었다 심폐소생술을 받은 암 환자도 125에서 163로 증가했다 김 교수는 코로나 환자도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다


/home/osung/anaconda3/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


464 283
time to summarize is 2.20798 sec
============ summary  ================
코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코코나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나가 확산으로 암 환자 등 일반 중 환자 치료에 공백이 생기고 있다는 목소리가 나온다. 김 교수는 코로나도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다. 코로나의 확산으로 일반 중환자의 치료가 공백이 생긴다는 목소리가 나오고 있다. 김교수는 코로나는 물론 이 때문에 방치된 환자들이 많다는 사실을 알아야 한다 말했다.
464 283
=============== long summary  ================
코로나 바이러스 확산으로 위중증 환자가 급격히 늘어나자 의료 인력이 코코나 치료에 집중하면서 일반 중환자 대처에 차질이 생기고 있다 의료 현장에서는 코로나가 확산으로 암 환자 등 일반 중 환자 치료에 공백이 생기고 있다는 목소리가 나온다. 김 교수는 코로나도 중요하지만 이 때문에 소외된 환자들이 방치되고 있다는 사실을 알아야 한다고 말했다. 코로나의 확산으로 일반 중환자의 치료가 공백이 생긴다는 목소리가 나오고 있다. 김교수는 코로나는 물론 이 때문에 방치된 환자들이 많다는 사실을 알아야 한다 말했다.
time to summarize long sentence is 1.32148 sec


In [28]:
run_summarize(df.iloc[1].text, tokenizer, model, device)

============ original ===============
 휴가때 골프 꼭 치고 싶으면 자비로 쳐라 허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 휴가 때 골프를 꼭 치고 싶은 사람은 문제가 되지 않을 사람과 자비로 쳐도 된다며 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다 청와대 관계자는 자비를 들여 취미로 치는 것까지 문제 삼을 수야 있겠느냐며 그렇다고 이제부터 골프를 쳐도 된다고 선언하는 것도 우습고 자연스럽게 분위기가 잡힐 것이라고 말했다 허 실장의 발언이 알려지자 관가에선 골프를 쳐도 된다는 건지 치지 말라는 건지 헷갈린다는 반응이 나왔다
497 317
time to summarize is 1.45685 sec
============ summary  ================
허태열 대통령비서실장이 최근 수석비서관회의를 주재하면서 휴가 때 골프를 꼭 치고 싶은 사람은 문제가 되지 않을 사람과 자비로 쳐도 된다며 웬만하면 필드골프장 대신 스크린골프를 이용하는 게 좋겠다고 말한 것으로 전해졌다 2월 박근혜 정부 출범 이후 골프 금지령이 내려진 적은 없지만 북한의 위협이 이어진 데다 정권 출범의 긴장감이 더해져 공직자들은 자연스레 골프를 칠 엄두를 내지 못했다 이와 관련해 박 대통령은 10일 언론사 논설실장 오찬에서 캐디들도 수입이 그렇고 자꾸 외국만 나가서 어떻게 하느냐고 걱정하는 이야기도 있다 지금 여러 가지 생각을 하고 있다고 밝혔다
497 317
=============== long summary  ================
허태열 대통령비서실장

In [30]:
df.columns

Index(['Unnamed: 0', 'id', 'topic', 'text', 'ksic', 'max_scores', 'keywords',
       'keywords_orig', 'ksic_scores', 'max_ksic', 'ksic_count',
       'max_ksic_scores'],
      dtype='object')

In [31]:
df.iloc[0]

Unnamed: 0                                                         0
id                                              NWRW2200000004.11879
topic                                                              P
text               코로나 환자 늘어 일반 중환자 치료 공백 코로나 바이러스 확산으로 위중증 환자가 급...
ksic                                                            c271
max_scores                                                    0.4621
keywords           {'김': 1, '분석': 1, '바이러스': 1, '코로나': 5, '병원': 2...
keywords_orig      {'환자': 11, '코로나': 5, '암': 5, '일반': 3, '중환자': 3...
ksic_scores        {'c102': 1, 'c272': 1, 'c211': 2, 'c271': 24, ...
max_ksic                                                    ['c271']
ksic_count                                                         1
max_ksic_scores                                                   24
Name: 0, dtype: object

In [66]:
df = df.drop(columns='Unnamed: 0')

In [33]:
df2.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores', 'keywords',
       'keywords_orig', 'ksic_scores', 'max_ksic', 'ksic_count',
       'max_ksic_scores'],
      dtype='object')

In [34]:
df = df2

In [79]:
df_sorted = df.sort_values(by='text', key=lambda x: x.str.len(), ascending=False)

In [81]:
df_sorted.iloc[0].text

"[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 되고, 그래서 이런 국면에서 필요한 것이 지난번에 합의했던 여야정 

In [86]:
len(df_sorted.iloc[0].text)

24234

In [90]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 5.79661 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길

902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360

===============  long summary2  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다.제가 분명하게 검찰에 말하고 싶은 것은 공수처도 수사권 조정법안도 그렇고, 지금까지 검찰이 사정기구로 본연의 역할을 다하지 못했기 때문에 개혁의 방안으로 논의가 되는 것이다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다. 민정수석이 가장 중요한 정부에서의 책임들이 인사검증과 권력기관들의 개혁이 제일 중요한 임무 가운데 하나다. 청와대 검증부터 청문회까지 전체가 하나의 검증 과정인 것이다.인사청문회가 정쟁쟁의 장으로 운영된다면 좋은 인사를 발탁하는 장이 아니라 좋은 인사를 막는 장이 될 것이다. 총리, 장관들이 정치에 나선다면 그것은 전적으로 본인 의사에 달려 있는 것이고, 대통령으로서 바라고 싶은 것은 선거에 나갈 생각이 있다면 선거시기에 임박해서가 아니라 충분한 여유를 두고 의사를 밝히는 것이 바람직하겠다고 생각하고 있다. 또한 요즘 유권자들의 요구도, 이제는 낙하산으로 공천 받아 내려오는 게 아니라 지역에서 지역주민과 밀착되길 바라기 때문에 그것이 유권자들 요구라고도 생각한다. 그뿐만 아

In [92]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 6.46697 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.검찰 스스로 개혁을 할 수 있는 많은 기회들을 지

In [94]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

============ original ===============
[전문] 문재인 정부 2년 특집 대담, 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 다음은 일문일답. ◆취임 2주년 소회 -2년 전 오늘 이 시간 기억하실지 모르겠다. 나라다운 나라를 만들어 달라, 이런 요구 속에서 문재인 정권이 출범을 했다. 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다. ▷먼저 국민들께 감사인사부터 드려야겠다. 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고, 저를 대통령으로 선택해 주셨다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 얼마나 기대에 부응했는지 잘 모르겠다. 많은 성과들이 있었다고 생각한다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다. ◆국내 정치 -국내 정치 현안 문제로 들어가야 할 것 같다. 조금 전에 대북 현안들, 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나. ▷이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다. 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다. -제가 여쭤본 것은 국내 정치권의…. ▷그렇다. 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만, 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다. 처리해야 할 민생법안도 많이 있고, 추경 문제도 논의해야 

24234 446
time to summarize is 6.44101 sec

============  summary  ================
문재인 대통령은 남북 정상회담에서 북한의 도발에 대해 대화와 협상을 통해 해결책을 찾겠다고 말했다. 대통령은 남북 정상회담이 성공적으로 개최될 수 있도록 최선을 다해야 한다고 말했다. 또한 남북이 대화의 장을 만들 수 있도록 노력해야 한다고 했다. 또한, 남북의 대화에 대한 대화의 장이 될 수 있도록 해야 한다. 또한 우리 정부는 남북의 평화적 협력의 길을 열어 나가야 한다고 생각한다. 또한 북한의 비핵화를 위해 노력해야 할 것이다. 또한 우리는 남북이 평화적으로 협력할 수 있는 길을 만들어 나가야 한다. -.,. ; 의 을 통해 남북의 길을 열어야 한다. 우리는 이 아니라 로 가는 길이다.의 길로 가야 한다.  남북의 길을 해 주길 바란다. 는데 라고 생각한다. 우리는 남북의 길로 나아가야 한다. 남북의 길의 길을 더 나아가야 한다고 생각한다며, 남북관계를 개선해야 한다. 우리 정부는 
1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262

===============  long summary  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길

902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360

===============  long summary2  ================
문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다.제가 분명하게 검찰에 말하고 싶은 것은 공수처도 수사권 조정법안도 그렇고, 지금까지 검찰이 사정기구로 본연의 역할을 다하지 못했기 때문에 개혁의 방안으로 논의가 되는 것이다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다. 민정수석이 가장 중요한 정부에서의 책임들이 인사검증과 권력기관들의 개혁이 제일 중요한 임무 가운데 하나다. 청와대 검증부터 청문회까지 전체가 하나의 검증 과정인 것이다.인사청문회가 정쟁쟁의 장으로 운영된다면 좋은 인사를 발탁하는 장이 아니라 좋은 인사를 막는 장이 될 것이다. 총리, 장관들이 정치에 나선다면 그것은 전적으로 본인 의사에 달려 있는 것이고, 대통령으로서 바라고 싶은 것은 선거에 나갈 생각이 있다면 선거시기에 임박해서가 아니라 충분한 여유를 두고 의사를 밝히는 것이 바람직하겠다고 생각하고 있다. 또한 요즘 유권자들의 요구도, 이제는 낙하산으로 공천 받아 내려오는 게 아니라 지역에서 지역주민과 밀착되길 바라기 때문에 그것이 유권자들 요구라고도 생각한다. 그뿐만 아

In [96]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

24234 446
1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411
1928 296
1928 345


In [59]:
ss = seperate_sentence(text, 1024)

In [60]:
ss

['',
 '전문 문재인 정부 2년 특집 대담 대통령에게 묻는다 문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS 문재인 정부 2년 특집 대담 대통령에게 묻는다에 출연해 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다 다음은 일문일답 취임 2주년 소회 2년 전 오늘 이 시간 기억하실지 모르겠다 나라다운 나라를 만들어 달라 이런 요구 속에서 문재인 정권이 출범을 했다 아무래도 첫 질문이니까 2년 동안 소회를 여쭙지 않을 수 없을 것 같다 먼저 국민들께 감사인사부터 드려야겠다 우리 국민들께서는 촛불혁명이라는 아주 성숙된 방법으로 정권을 교체하고 저를 대통령으로 선택해 주셨다 그래서 문재인 정부는 촛불정신 위에 서 있다 촛불민심이 명하는 대로 국정농단 그리고 반칙과 특권이라는 적폐시대를 마감하고 새로운 시대 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다 얼마나 기대에 부응했는지 잘 모르겠다 많은 성과들이 있었다고 생각한다 그러나 한편으로 또 아쉬운 부분들도 많이 있고 또 보완해야 할 과제들도 많이 있다고 느낀다 앞으로 그 점을 더 집중해서 국민들이 바라는 그런 나라를 만들도록 노력하겠다 국내 정치 국내 정치 현안 문제로 들어가야 할 것 같다 조금 전에 대북 현안들 그리고 식량지원문제를 화두로 한 번 만나보자 제안을 공식적으로 하신 거라 봐야 하나 이미 남북 간 정상회담을 가져야겠다라는 것은 제가 공개적으로 이미 발언을 했다 실제로 북한하고 실무적인 대화에 들어가겠다는 뜻을 말씀 드렸다 제가 여쭤본 것은 국내 정치권의 그렇다 지금 패스트트랙 문제로 여야 정치권이 이렇게 대치하고 있는 것은 정치 성격상 우리가 이해할 수 있는 것이지만 국민들 입장에서 볼 때는 참으로 답답한 국면이 아닐 수 없다 처리해야 할 민생법안도 많이 있고 추경 문제도 논의해야 되고 그래서 이런 국면에서 필요한 것이 지난번에 합의했던 여야정 상설국정협의체를 가동하는 것이라고 생각한다 우선적으로 방금 현안으로 대두된

In [61]:
len(ss)

2

In [100]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=2048, verify=False)

1993 285
2026 277
2024 244
2019 366
1994 268
1985 324
2020 343
2043 350
2024 327
2030 351
2034 353
1598 411
1928 296
1928 345


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.한일 관계문제다. 과거사 문제는 과거를 지울 수 없는 문제고 과거사 문제가 한일관계의 족쇄가 된 게 너무 오랜 시간이다. 다만, 어려움을 겪고 있는 것은 과거사 문제로 한번씩 양국관계발전에 발목을 잡고 있는데 그것은 결코 한국정부가 만들어 내고 있는 문제가 아니다. 하지만 일본 정치지도자들이 자꾸 그 문제를 국내 정치적인 문제로 다루기 때문에 미래지향적인 발전의 발목을 잡는 일이 거듭되고 있다고 본다. -제일 문제는 청년일자리다. 제조업의 혁신해 고도화해서 경쟁력을 높여 일자리를 늘리는 방향이 하나 있고, 신산업들을 빨리 성장시켜서 좋은 일자리를 만들어야겠고, 요즘 벤처창업들을 크게 늘리고 지원할 필요가 있다."

In [101]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1024, verify=False)

1002 344
991 321
972 314
991 317
973 283
882 331
919 307
986 321
1012 292
972 301
996 323
1010 332
1006 321
1006 319
926 312
998 448
957 373
997 280
961 315
986 306
958 288
1012 290
964 312
971 436
342 262
902 281
936 307
936 351
928 366
1014 342
1017 315
1008 398
940 313
279 360
997 396
991 338
1018 317
1043 311


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다."

In [102]:
run_summarize(df_sorted.iloc[0].text, tokenizer, model, device, chunk=1536, verify=False)

1517 285
1448 304
1510 269
1505 232
1466 377
1489 361
1399 331
1477 383
1419 354
1414 402
1509 315
1497 362
1493 339
1488 334
1526 283
1633 411
1395 416
1384 392
1358 328
1153 333


"문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한

In [103]:
t = "문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 문재인 대통령은 이날 대담은 예정된 시간을 6분 넘겨 86분간 생중계로 진행됐다. 그러나 한편으로 또 아쉬운 부분들도 많이 있고, 또 보완해야 할 과제들도 많이 있다고 느낀다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다."

In [105]:
len(t)

311

In [106]:
len("문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.한일 관계문제다. 과거사 문제는 과거를 지울 수 없는 문제고 과거사 문제가 한일관계의 족쇄가 된 게 너무 오랜 시간이다. 다만, 어려움을 겪고 있는 것은 과거사 문제로 한번씩 양국관계발전에 발목을 잡고 있는데 그것은 결코 한국정부가 만들어 내고 있는 문제가 아니다. 하지만 일본 정치지도자들이 자꾸 그 문제를 국내 정치적인 문제로 다루기 때문에 미래지향적인 발전의 발목을 잡는 일이 거듭되고 있다고 본다. -제일 문제는 청년일자리다. 제조업의 혁신해 고도화해서 경쟁력을 높여 일자리를 늘리는 방향이 하나 있고, 신산업들을 빨리 성장시켜서 좋은 일자리를 만들어야겠고, 요즘 벤처창업들을 크게 늘리고 지원할 필요가 있다.")

641

In [107]:
len("문재인 대통령은 9일 오후 청와대 상춘재에서 진행된 KBS '문재인 정부 2년 특집 대담, 대통령에게 묻는다'에 출연해, 정부 출범 2주년 소회 및 각 분야별 입장을 밝혔다. 그래서 문재인 정부는 촛불정신 위에 서 있다. 촛불민심이 명하는 대로 국정농단, 그리고 반칙과 특권이라는 적폐시대를 마감하고, 새로운 시대, 공정하고 정의로운 대한민국의 길을 향해서 걸어가고 있다.그래서 그 해법을 선택한 것을 가지고 독재라 하는 것은 정말 조금 맞지 않는 얘기란 말씀을 드린다. 검찰 스스로 개혁을 할 수 있는 많은 기회들을 지금까지 놓쳐 왔다. 그래서 검찰은 개혁의 당사자이고, 말하자면 이제는 셀프개혁으로서는 안 된다는 게 국민들의 보편적인 생각이기 때문에 그런 방안들이 마련되고 있는 것이어서 검찰이 보다 좀 겸허한 자세를 가져야한다고 생각한다.북한이 며칠 전에 여러 종류의 단거리 발사체를 발사한 데 이어서 오늘은 일단 단거리 미사일로 추정되는 그런 발사를 했다. 유엔 안보리 결의 속에는 탄도미사일을 하지 말라는 그런 표현이 있기 때문에 비록 단거리라 할지라도 그것이 탄도미사일일 경우에는 유엔안보리 결의에 위반될 소지도 없지 않다고 생각한다. 그런 방식으로 북한의 의도를 여러 가지로 해석하게 만들고 또 우려하게 만들고 자칫 잘못하면 대화와 협상국면의 찻물을 끼얹는 이런 성격을 거듭 하는 것은 결코 바람직하지 않다는 것을 북한측에 다시 한 번 이야기하고 싶다. 그런 형편이기 때문에 북한동포들의 심각한 기아상태를 우리가 외면할 수 없고, 우리가 동포애나 인도주의적인 차원에서라도 우리가 북한의 식량을 지원할 필요가 있다고 생각하는 것이다.북한의 식량지원 문제에 대해서 저로서는 가장 바람직한 것은 지금 패스트트랙 문제 때문에 여야 간 정국이 교착상태에 빠져 있는데 그 문제는 별도로 해결하더라도 북한에 대한 식량지원에 대해서는 대통령과 여야가 함께 모여서 협의를 하는 것이 바람직하지 않을까 라는 생각을 하고 있다. 그래서 그렇기 때문에 사실은 식량지원의 문제에 대해서 우리가 한미간에 합의를 한 것이 이번 발사 이전인데, 그 이후에 또 다시 발사가 있었기 때문에 이 점에 대해서는 국민들의 공감이나 제재가 필요하다고 생각한다. 한일관계는 굉장히 중요하다고 생각하고, 앞으로 더 미래지향적으로 발전돼 나가야 한다고 생각한다.고령화가 급격하게 진행되고 있어서 65세 이상 인구가 14%가 넘는 고령사회를 이미 2017년에 통과했고, 2025년이면 20%가 넘는 초고령사회가 될 것으로 예상된다. 고령인구가 크게 늘고 있기 때문에 일자리 수를 더 늘리고 과거의 급여가 낮았기 때문에 급여를 두배 높여서 실제로 어르신들의 빈곤 해결에 도움되도록 정부가 노력하고 있는 것이다. 거시적인 경제성공은 인정하고 자부심을 가져야 한다고 생각한다. 다만 국민들에게 고르게 다 소득배분이 되고 있지 않고 있기 때문에 아직도 양극화가 심각하고 특히 소득이 낮은 층의 소득이 늘지 않고 있기때문에 그분들의 사는 문제가 해결이 안 되고 있다.")

1469

In [109]:
filename2='/home/osung/data/korean/modu/json/combined_news.tsv'

In [115]:
df2 = pd.read_csv(filename2, sep='\t')

In [116]:
len(df2)

3611424

In [112]:
df2.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')

In [113]:
len(df)

1530052

In [114]:
df.columns

Index(['id', 'topic', 'text', 'ksic', 'max_scores'], dtype='object')